In [ ]:



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.feature_engineering import FeatureEngineer

# Load cleaned data
df = pd.read_csv('../data/processed/cleaned_data.csv')
print(f"Loaded data shape: {df.shape}")

# Initialize feature engineer
engineer = FeatureEngineer()

# Create new features
df_features = engineer.create_features(df)
print(f"Data shape after feature engineering: {df_features.shape}")
print(f"\nNew features added: {list(set(df_features.columns) - set(df.columns))}")

# Display new features statistics
new_features = ['like_follower_ratio', 'post_follower_ratio', 
                'avg_likes_ratio', 'engagement_efficiency', 
                'posts_per_million_followers']

print("\nStatistics of New Features:")
print(df_features[new_features].describe())

# Visualize new features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(new_features):
    df_features[feature].hist(ax=axes[i], bins=30, edgecolor='black')
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Correlation of new features with target
target_correlation = df_features.corr()['influence_score'].sort_values(ascending=False)
print("\nCorrelation with Influence Score:")
print(target_correlation.head(10))

# Prepare data for modeling
X_reg, y_reg = engineer.prepare_regression_features(df_features)
print(f"\nRegression Features shape: {X_reg.shape}")
print(f"Regression Target shape: {y_reg.shape}")

# Prepare for classification
df_class = df_features.copy()
df_class['engagement_rate_class'] = pd.cut(
    df_class['60_day_eng_rate'],
    bins=[0, 1, 3, df_class['60_day_eng_rate'].max()],
    labels=['Low', 'Medium', 'High']
)

print("\nEngagement Rate Class Distribution:")
print(df_class['engagement_rate_class'].value_counts())

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_class['country_encoded'] = encoder.fit_transform(df_class['country'].fillna('Unknown'))

# Prepare classification features
X_cls = df_class[['followers', 'influence_score', 'country_encoded']]
y_cls = df_class['engagement_rate_class']

print(f"\nClassification Features shape: {X_cls.shape}")
print(f"Classification Target shape: {y_cls.shape}")

# Save features for modeling
df_features.to_csv('../data/processed/data_with_features.csv', index=False)
df_class.to_csv('../data/processed/data_for_classification.csv', index=False)

print("\nFeature engineered data saved to:")
print("- '../data/processed/data_with_features.csv'")
print("- '../data/processed/data_for_classification.csv'")